In [1]:
import os
import sys
import time
import json
from datetime import datetime

import numpy as np
import pandas as pd

#from models import models
from utils import read_df, read_numpy, write_df
import logging

from evaluate import rps

In [2]:
#Model imports
import numpy as np
import json
import os
import pandas as pd

from tqdm.auto import tqdm
from multiprocessing import Pool
from iteround import saferound
import scipy.stats as st
import tweedie
from category_encoders import OrdinalEncoder
from pandarallel import pandarallel
pandarallel.initialize(nb_workers=50)

from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error

from model import Model
from utils import read_df

INFO: Pandarallel will run on 50 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
DATASET_PATH = './dataset/processed/train_v1'
TEST_DATA_FILENAME = 'test_data.csv'
GROUND_TRUTH_FILENAME = 'test_ground_truth.npy'
TRAIN_DATA_FILENAME = 'train_data.parquet'
TRAIN_DATA_PROCESSED_FILENAME = 'train_data_features.parquet'
ITEM_DATA_FILEPATH = './dataset/items_static_metadata_full.jl'

TRAIN_DATA_X_FILENAME = 'train_data_x.parquet'
TRAIN_DATA_Y_FILENAME = 'train_data_y.parquet'
TRAIN_DATA_X_PROCESSED_FILENAME = 'train_data_x_features.parquet'
TRAIN_DATA_Y_PROCESSED_FILENAME = 'train_data_y_features.parquet'

TEST_FROMTRAIN_DATA_LAST29_FILENAME = 'test_fromtrain_data_last29.parquet'
TEST_FROMTRAIN_DATA_LAST29_PROCESSED_FILENAME = 'test_fromtrain_data_last29_features.parquet'

TRAIN_DATA_X_PROCESSED_TSFRESH_FILENAME = 'train_data_x_features_tsfresh.parquet'
TEST_FROMTRAIN_DATA_LAST29_PROCESSED_TSFRESH_FILENAME = 'test_fromtrain_data_last29_features_tsfresh.parquet'

TRAIN_DATA_PROCESSEDV2_FILENAME = 'train_data_featuresv2.parquet'
TRAIN_DATA_X_PROCESSEDV2_FILENAME = 'train_data_x_featuresv2.parquet'
TEST_FROMTRAIN_DATA_LAST29V2_FILENAME = 'test_fromtrain_data_last29_featuresv2.parquet'

In [4]:
dateset_indexes = [0]
model_name = 'simple_first_30_days_fixed_spike'

In [5]:
dataset_index = dateset_indexes[0]
dataset_current_path = os.path.join(DATASET_PATH, str(dataset_index))

test_data_filepath = os.path.join(dataset_current_path, TEST_DATA_FILENAME)
ground_truth_filepath = os.path.join(dataset_current_path, GROUND_TRUTH_FILENAME)
train_data_filepath = os.path.join(dataset_current_path, TRAIN_DATA_FILENAME)
train_data_processed_filepath = os.path.join(dataset_current_path, TRAIN_DATA_PROCESSED_FILENAME)

train_data_x_filepath = os.path.join(dataset_current_path, TRAIN_DATA_X_FILENAME)
train_data_y_filepath = os.path.join(dataset_current_path, TRAIN_DATA_Y_FILENAME)
train_data_x_processed_filepath = os.path.join(dataset_current_path, TRAIN_DATA_X_PROCESSED_FILENAME)
train_data_y_processed_filepath = os.path.join(dataset_current_path, TRAIN_DATA_Y_PROCESSED_FILENAME)

test_fromtrain_data_last29_filepath = os.path.join(dataset_current_path, TEST_FROMTRAIN_DATA_LAST29_FILENAME)
test_fromtrain_data_last29_processed_filepath = os.path.join(dataset_current_path, TEST_FROMTRAIN_DATA_LAST29_PROCESSED_FILENAME)

train_data_x_processed_tsfresh_filepath = os.path.join(dataset_current_path, TRAIN_DATA_X_PROCESSED_TSFRESH_FILENAME)
test_fromtrain_data_last29_processed_tsfresh_filepath = os.path.join(dataset_current_path, TEST_FROMTRAIN_DATA_LAST29_PROCESSED_TSFRESH_FILENAME)

train_data_processedv2_filepath = os.path.join(dataset_current_path, TRAIN_DATA_PROCESSEDV2_FILENAME)
train_data_x_processedv2_filepath = os.path.join(dataset_current_path, TRAIN_DATA_X_PROCESSEDV2_FILENAME)
test_fromtrain_data_last29_processedv2_filepath = os.path.join(dataset_current_path, TEST_FROMTRAIN_DATA_LAST29V2_FILENAME)

In [6]:
#model = models['xgboost_v1']('./dataset')
#model.prepare_data()
#model.train()
#model.predict('./dataset/test_data.csv')

In [7]:
#df_train = read_df(train_data_filepath)
#df_item = read_df(ITEM_DATA_FILEPATH)

In [8]:
df_test = read_df(test_data_filepath)
ground_truth = read_numpy(ground_truth_filepath)

#df_train_processed = read_df(train_data_processed_filepath)

In [9]:
#df_train_x = read_df(train_data_x_filepath)
#df_train_x = read_df(train_data_y_filepath)

#df_test_fromtrain_x_features = read_df(test_fromtrain_data_last29_processed_filepath)
#df_train_x_features = read_df(train_data_x_processed_filepath)
df_train_y_features = read_df(train_data_y_processed_filepath)

In [10]:
#df_train_x_features_tsfresh = read_df(train_data_x_processed_tsfresh_filepath)
#df_test_fromtrain_x_features_tsfresh = read_df(test_fromtrain_data_last29_processed_tsfresh_filepath)

df_train_featuresv2 = read_df(train_data_processedv2_filepath)
df_train_x_featuresv2 = read_df(train_data_x_processedv2_filepath)
df_test_fromtrain_x_featuresv2 = read_df(test_fromtrain_data_last29_processedv2_filepath)

In [11]:
df_train_featuresv2['item_id'].cat.categories

Index(['184358', '150336', '288936', '502053', '200460', '268926', '388726',
       '197622', '345164', '122043',
       ...
       '349358', '158690', '432635', '94776', '485451', '80319', '340085',
       '37558', '394663', '254647'],
      dtype='object', length=517896)

In [12]:
df_test_fromtrain_x_featuresv2['item_id'].cat.categories

Index(['184358', '150336', '288936', '502053', '200460', '268926', '388726',
       '197622', '345164', '122043',
       ...
       '349358', '158690', '432635', '94776', '485451', '80319', '340085',
       '37558', '394663', '254647'],
      dtype='object', length=517896)

In [13]:
id_column =  'sku'
date_column = 'date'

item_string_columns = ['item_title']
item_categorical_columns = ['item_domain_id', 'item_id', 'site_id', 'product_id', 'product_family_id']
item_columns = item_string_columns + item_categorical_columns

sku_numeric_columns = ['sold_quantity', 'current_price', 'minutes_active']
sku_categorical_columns = ['currency', 'listing_type', 'shipping_logistic_type', 'shipping_payment']
sku_columns = sku_numeric_columns + sku_categorical_columns

string_columns = item_string_columns
categorical_columns = sku_categorical_columns + item_categorical_columns
numeric_columns = sku_numeric_columns

In [14]:
df_train_x_featuresv2

,item_title,item_domain_id,item_id,site_id,product_id,product_family_id,sku,count,date__first,date__last,...,minutes_active__by_item_domain_id__by_dayofthemonth__with_smaller_sum,minutes_active__by_item_domain_id__by_dayofthemonth__with_smaller_mean,minutes_active__by_item_domain_id__by_dayofthemonth__bigger_std,minutes_active__by_item_domain_id__by_dayofthemonth__bigger_var,minutes_active__by_item_domain_id__by_dayofthemonth__smaller_std,minutes_active__by_item_domain_id__by_dayofthemonth__smaller_var,minutes_active__by_item_domain_id__by_dayofthemonth__with_bigger_std,minutes_active__by_item_domain_id__by_dayofthemonth__with_bigger_var,minutes_active__by_item_domain_id__by_dayofthemonth__with_smaller_std,minutes_active__by_item_domain_id__by_dayofthemonth__with_smaller_var
index,,,,,,,,,,,,,,,,,,,,,
0,Suporte Rack Prancha Parede C/ Regulagem Horiz...,MLB-SURFBOARD_RACKS,300279,MLB,nan,nan,1,29,2021-02-01,2021-03-01,...,19,19,747.179311,558276.923077,399.384141,159507.692308,3,3,27,27
1,Suporte Rack Prancha Parede C/ Regulagem Horiz...,MLB-SURFBOARD_RACKS,300279,MLB,nan,nan,109871,29,2021-02-01,2021-03-01,...,19,19,747.179311,558276.923077,399.384141,159507.692308,3,3,27,27
2,Suporte Rack De Parede Prancha De Surf Modelo ...,MLB-SURFBOARD_RACKS,447787,MLB,nan,nan,119816,29,2021-02-01,2021-03-01,...,19,19,747.179311,558276.923077,399.384141,159507.692308,3,3,27,27
3,Rack Parede Prancha Surf Longboard Fun Suporte...,MLB-SURFBOARD_RACKS,258668,MLB,nan,nan,270493,29,2021-02-01,2021-03-01,...,19,19,747.179311,558276.923077,399.384141,159507.692308,3,3,27,27
4,Suporte Rack Parede Prancha Surf Stand Up Long...,MLB-SURFBOARD_RACKS,224549,MLB,nan,nan,351784,29,2021-02-01,2021-03-01,...,19,19,747.179311,558276.923077,399.384141,159507.692308,3,3,27,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495542,Nozzle Original Creality 3d Ender 3 Y Pro - Pi...,MLA-3D_PRINTER_NOZZLES,470915,MLA,nan,nan,655637,29,2021-02-01,2021-03-01,...,1,1,0.000000,0.000000,0.000000,0.000000,1,1,1,1
495543,Trailer Maletín Transportador De Autos Carrito...,MLM-COLLECTION_DISPLAYS,137413,MLM,nan,nan,655681,29,2021-02-01,2021-03-01,...,26,26,0.000000,0.000000,0.000000,0.000000,1,1,1,1
495544,Moedor De Cafe Eletrico Ceramica Portatil Rec...,MLB-ELECTRIC_GRAIN_MILLS,169122,MLB,nan,nan,656129,29,2021-02-01,2021-03-01,...,8,8,0.000000,0.000000,0.000000,0.000000,1,1,1,1


In [15]:
isna = df_train_x_featuresv2.isna().sum()
isna[isna > 0]

Series([], dtype: int64)

In [16]:
x_df = df_train_x_featuresv2.set_index('sku').sample(frac=1, random_state=42).copy()
#x_df = df_test_fromtrain_x_features_tsfresh.sample(frac=1, random_state=42).reset_index(drop=True).copy()

In [17]:
y_df = df_train_y_features.set_index('sku').loc[x_df.index].copy()
#y_df = df_train_y_features.set_index('sku').loc[x_df['sku']].reset_index().copy()

In [18]:
features = []
for feature in x_df.columns:
    if 'by_item_domain_id' in feature:
        continue
    if 'by_dayofthemonth' in feature or 'by_dayoftheweek' in feature or 'by_weekofthemonth' in feature:
        continue
    else:
        features.append(feature)

In [19]:
from tsfresh.utilities.dataframe_functions import impute

In [20]:
def target_stock_features(df):
    df['target_stock__target_day_by_mean'] = df['target_stock']/df['sold_quantity__mean']
    df['target_stock__target_day_by_mean__std'] = df['sold_quantity__std']*df['target_stock__target_day_by_mean']/df['sold_quantity__mean']
    df['target_stock__last_5__target_day_by_mean'] = df['target_stock']/df['sold_quantity__last_5__mean']
    df['target_stock__last_5__target_day_by_mean__std'] = df['sold_quantity__last_5__std']*df['target_stock__last_5__target_day_by_mean']/df['sold_quantity__last_5__mean']
    
    #df['target_stock__by_item_domain_id__target_day_by_mean'] = df['target_stock']/df['sold_quantity__by_item_domain_id__mean']
    #df['target_stock__by_item_domain_id__target_day_by_mean__std'] = df['sold_quantity__by_item_domain_id__std']*df['target_stock__by_item_domain_id__target_day_by_mean']/df['sold_quantity__by_item_domain_id__mean']
    
    #df.replace([np.inf, -np.inf], np.nan, inplace=True)
    for feature in df.columns:
        if feature.startswith('target_stock__'):
            df[feature] = impute(df[[feature]].copy())[feature]/30

In [21]:
def weak_normalize(df, normalize_positional=True, one_hot_categorical=False, one_hot_positional=False):
    categories_to_eliminate = ['item_domain_id', 'item_id', 'product_id', 'product_family_id']
    counting_to_eliminate = ['count']
    positional_to_eliminate = ['date__last_month', 'date__first_month', 'date__diff', 'date__last_weekofmonth', 'date__last_day', 'date__last_dayofweek']
    #numerical_to_eliminate = ['__sum', '']
    
    numeric_features = []
    categorical_features = []
    positional_features = []
    counting_features = []
    date_features = []
    string_features = []
    series_features = []
    id_features = []
    for feature in df.columns:
        feature_components = feature.split('__')
        if ('with' in feature_components[-1]) or ('location' in feature_components[-1]):
            positional_features.append(feature)
        elif ('count' in feature_components[-1]):
            counting_features.append(feature)
        elif feature_components[-1] == 'series':
            series_features.append(feature)
        elif feature_components[0] in categorical_columns:
            categorical_features.append(feature)
        elif feature_components[0] == 'date':
            if feature == 'date__first' or feature == 'date__last':
                date_features.append(feature)
            else:
                positional_features.append(feature)
        elif feature_components[0] in string_columns:
            string_features.append(feature)
        elif feature_components[0] == id_column:
            id_features.append(feature)
        else:
            numeric_features.append(feature)
    
    numeric_features_tmp = []
    categorical_features_tmp = []
    positional_features_tmp = []
    counting_features_tmp = []
    #target_stock_features = ['target_stock']
    
    #for feature in df.columns:
    #    if feature.startswith('target_stock__'):
    #        df[feature] = df[feature]/30
    #        target_stock_features.append(feature)
    
    for feature in categorical_features:
        feature_components = feature.split('__')
        if feature_components[0] not in categories_to_eliminate:
            categorical_features_tmp.append(feature)
            if one_hot_categorical:
                pass #TODO implementar onehot
            else:
                df[feature] = df[feature].cat.codes
                
    for feature in counting_features:
        feature_components = feature.split('__')
        if feature_components[0] not in counting_to_eliminate:
            if 'mode' in feature_components[-1]:
                count_feature_base = df['count'] if feature_components[1] != 'by_item_domain_id' else df['count__by_item_domain_id']
            else:
                count_feature_base = "__".join(feature_components[:-1])
                count_feature_base = df[count_feature_base+'__count_of_zero'] + df[count_feature_base+'__count_of_non_zero']
            df[feature] = df[feature]/count_feature_base
            df[feature] = df[feature].fillna(0)
            counting_features_tmp.append(feature)
    
    for feature in positional_features:
        if feature in positional_to_eliminate:
            continue
        feature_components = feature.split('__')
        is_count_based = False
        if 'dayoftheweek' in feature_components[-2] or 'dayofweek' in feature_components[-1]:
            sub = 0
            positional_base = 7
        elif 'weekofthemonth' in feature_components[-2] or 'weekofmonth' in feature_components[-1]:
            sub = 0
            positional_base = 4
        elif 'dayofthemonth' in feature_components[-2] or 'day' in feature_components[-1]:
            sub = 1
            positional_base = 31
        else:
            is_count_based = True
            sub = 0
            positional_base = df['count']
            
        if normalize_positional:
            if one_hot_positional:
                pass #TODO implementar onehot
            else:
                df[feature] = (df[feature] - sub)/positional_base
                positional_features_tmp.append(feature)
        else:
            if is_count_based:
                df[feature] = df[feature].apply(lambda x: x if x >= 0 else np.nan)
                df[feature] = df[feature]-df['count']+29
                df[feature] = df[feature].fillna(-31)
            positional_features_tmp.append(feature)
            
    for feature in numeric_features:
        feature_components = feature.split('__')
        if 'linregress' in feature_components[-1] or 'energy_ratio' in feature_components[-1]:
            numeric_features_tmp.append(feature)
        elif 'variance_large_than_std' == feature_components[-1] or 'symmetry_looking' in feature_components[-1]:
            df[feature] = df[feature].astype(int)
            numeric_features_tmp.append(feature)
        elif feature_components[-1] == 'sum' or feature_components[-1] == 'abs_energy':
            #if 'by' not in feature_components[-2] and 'last' not in feature_components[-2]  and 'change' not in feature_components[-2]:
            #    df[feature] = df[feature]*29/df['count']    
            if 'change' in feature_components[-2]:
                df[feature] = df[feature]*28/(df['count']-1)
                df[feature] = df[feature].fillna(0)
            elif 'last' in feature_components[-2]:
                numeric_features_tmp.append(feature)
        else:
            numeric_features_tmp.append(feature)
            
    features = numeric_features_tmp + categorical_features_tmp + positional_features_tmp + counting_features_tmp #+ target_stock_features
    return df[features]

28 - 29 + 29 =28
0 - 1 + 29 = 28
0 - 59 +29 = -30
58 - 59 +29 = 28
-1 -1 + 29 = 27
-1 -29 +29 = -1
-1 -59 +29 = -31


x_df_filtered_return_mean = select_features(x_df, pd.Series(return_mean, index=y_df.index), n_jobs=96)
x_df_filtered_td = select_features(x_df, pd.Series(y_td, index=y_df.index), n_jobs=96)
x_df_filtered_sum = select_features(x_df, y_df['sold_quantity_sum'], n_jobs=96)
x_df_filtered_mean = select_features(x_df, y_df['sold_quantity_mean'], n_jobs=96)

filtered_dfs = {
    'x_df_filtered_return_mean': x_df_filtered_return_mean,
    'x_df_filtered_td': x_df_filtered_td,
    'x_df_filtered_sum': x_df_filtered_sum,
    'x_df_filtered_mean': x_df_filtered_mean
}

for name in filtered_dfs:
    with open(os.path.join(dataset_current_path,name+'.txt'), 'w', encoding='utf8') as f:
        for column in filtered_dfs[name].columns:
            f.write(column+'/n')

In [22]:
x_df_train = x_df[features].copy()
#x_df_train['target_stock'] = y_ts
#x_df_train = x_df_train[features]
#target_stock_features(x_df_train)
x_df_train = weak_normalize(x_df_train)
X = x_df_train.values

In [23]:
#x_normalized = x_df_train.copy()
#x_normalized = weak_normalize(x_normalized)

In [24]:
removed_fetures = set(features) - set(x_df_train.columns)
len(removed_fetures)

39

In [25]:
x_test_df = df_test_fromtrain_x_featuresv2.set_index('sku').loc[df_test['sku']].copy()
x_test_df = x_test_df[features]
#x_test_df = x_test_df[features]
x_test_df = weak_normalize(x_test_df)
x_test_df['target_stock'] = df_test.set_index('sku')['target_stock']
target_stock_features(x_test_df)
X_test = x_test_df.values

In [26]:
x_test_df_all = df_train_featuresv2.set_index('sku').loc[df_test['sku']][features].copy()
#x_test_df_all = x_test_df_all[features]
x_test_df_all = weak_normalize(x_test_df_all)
x_test_df_all['target_stock'] = df_test.set_index('sku')['target_stock']
target_stock_features(x_test_df_all)
X_test_all = x_test_df_all.values

In [27]:
import time
from matplotlib import pyplot as plt
def print_feature_importance(importance_array, feature_names, n=30):
    fig, ax = plt.subplots(figsize=(20, 10))
    sort = np.flip(np.argsort(importance_array))[0:n]
    ax.barh(np.array(feature_names)[sort], importance_array[sort])
    plt.show()

In [28]:
from xgboost import XGBClassifier

In [29]:
y_sold_quantity_series = np.array(list(y_df['sold_quantity_series'].apply(json.loads).values))
y_cumsum = y_sold_quantity_series.cumsum(axis=1)

In [30]:
SELECT_N = 31

index_list = []
y_ts = []
y_td = []

for i, t in zip(y_df.index, y_cumsum):
    target_stocks, target_dates = np.unique(t, return_index=True)
    if target_stocks[0] == 0:
        target_stocks = target_stocks[1:]
        target_dates = target_dates[1:]
    size = len(target_stocks)
    index = np.arange(size)
    if size > SELECT_N:
        np.random.shuffle(index)
        index = index[:SELECT_N]

    for target_stock, target_date in zip(target_stocks[index], target_dates[index]):
        index_list.append(i)
        y_ts.append(target_stock)
        y_td.append(target_date)

index_list = np.array(index_list)
y_ts = np.array(y_ts)
y_td = np.array(y_td)

x_df_train_2 = x_df_train.loc[index_list].copy()
x_df_train_2['target_stock'] = y_ts
target_stock_features(x_df_train_2)
X = x_df_train_2.values

In [31]:
y = y_td

In [32]:
y.shape

(3953690,)

In [33]:
i = 29
y_day = y.copy()
index_y_day = (y_day == i).nonzero()[0]
index_y_not_day = (y_day != i).nonzero()[0]
np.random.shuffle(index_y_not_day)
index_y_not_day = index_y_not_day[:index_y_day.shape[0]]

In [34]:
y_day = np.concatenate((np.ones(index_y_day.shape[0]), np.zeros(index_y_not_day.shape[0])))
X_day = X[np.concatenate((index_y_day, index_y_not_day))].copy()

In [35]:
p = np.random.permutation(X_day.shape[0])

In [36]:
y_day = y_day[p]
X_day = X_day[p]

In [37]:
y_true_all = ground_truth.argmax(axis=1)
y_true_on_day = (y_true_all == i).nonzero()[0]
y_true_not_on_day = (y_true_all != i).nonzero()[0]
y_true_not_on_day = y_true_not_on_day[:y_true_on_day.shape[0]]

y_test_day = np.concatenate((np.ones(y_true_on_day.shape[0]), np.zeros(y_true_not_on_day.shape[0])))
X_test_day = X_test_all[np.concatenate((y_true_on_day, y_true_not_on_day))].copy()

In [38]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_day, y_day, test_size=0.2, random_state=42)

In [39]:
model_name = f'day_{i}'
gpu_id = 2#i%3 + 1
print(model_name, "Starting model training on gpu", gpu_id)
start_time = time.time()
model = XGBClassifier(n_estimators=100, max_depth=10, learning_rate=0.2,
                    random_state=0, tree_method='gpu_hist', objective='binary:logistic',
                    gpu_id=gpu_id, use_label_encoder=False)
model.fit(X_train, y_train)
print(model_name, "Model Training took:", str(time.time() - start_time), "seconds")

day_29 Starting model training on gpu 3
[10:21:50] WARNING: /home/edugarcia/dependencies/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
day_29 Model Training took: 7.941522121429443 seconds


In [40]:
from sklearn.metrics import f1_score
y_pred = model.predict(X_val)
f1_score(y_val, y_pred)

0.6949182469894408

In [42]:
y_pred = model.predict(X_test_day)
f1_score(y_test_day, y_pred)

0.6229972559086484

In [45]:
y_pred_all = model.predict(X_test_all)
f1_score((y_true_all == i).astype(int), y_pred_all)

0.12946781700115892

# Train for each day

In [216]:
def train_model(day):
    i = day

    y_day = y.copy()
    index_y_day = (y_day == i).nonzero()[0]
    index_y_not_day = (y_day != i).nonzero()[0]
    np.random.shuffle(index_y_not_day)
    index_y_not_day = index_y_not_day[:index_y_day.shape[0]]
    
    y_day = np.concatenate((np.ones(index_y_day.shape[0]), np.zerps(index_y_not_day.shape[0])))
    X_day = X[np.concatenate((index_y_day, index_y_not_day))].copy()
    
    p = np.random.permutation(X_day.shape[0])
    
    y_day = y_day[p]
    X_day = X_day[p]
    
    model_name = f'day_{i}'
    gpu_id = i%3 + 1
    print(model_name, "Starting model training on gpu", gpu_id)
    start_time = time.time()
    model = XGBClassifier(n_estimators=100, max_depth=10, learning_rate=0.1,
                        random_state=0, tree_method='gpu_hist', objective='binary:logistic',
                        gpu_id=gpu_id, use_label_encoder=False)
    model.fit(X_day, y_day)
    print(model_name, "Model Training took:", str(time.time() - start_time), "seconds")

    return {'day': i, 'model': model}

In [217]:
from multiprocessing.dummy import Pool as ThreadPool

result_data = []
with ThreadPool(3) as p:
    for data in tqdm(p.imap(train_model, range(30)), total=30):
        result_data.append(data)

  0%|          | 0/30 [00:00<?, ?it/s]

day_2 Starting model training on gpu 3
day_1 Starting model training on gpu 2
day_0 Starting model training on gpu 1
day_0 Model Training took: 30.04883885383606 seconds
day_2 Model Training took: 31.280530214309692 seconds
day_0 Model Training took: 13.989392042160034 seconds
day_2 Model Training took: 14.985158205032349 seconds
day_3 Starting model training on gpu 1
day_4 Starting model training on gpu 2
day_1 Model Training took: 32.12942099571228 seconds
day_1 Model Training took: 22.626384973526 seconds
day_3 Model Training took: 7.9581458568573 seconds
day_5 Starting model training on gpu 3
day_6 Starting model training on gpu 1
day_4 Model Training took: 11.096652269363403 seconds
day_7 Starting model training on gpu 2
day_5 Model Training took: 8.067754983901978 seconds
day_6 Model Training took: 7.705057144165039 seconds
day_8 Starting model training on gpu 3
day_9 Starting model training on gpu 1
day_7 Model Training took: 7.866790771484375 seconds
day_10 Starting model train

In [224]:
preds = np.zeros((X_test_all.shape[0], 30))
for model_data in result_data:
    model = model_data['model']
    day = model_data['day']
    pred_day = model.predict_proba(X_test_all)
    argmax_day = pred_day.argmax(axis=1)
    print(day, argmax_day.sum()/len(pred_day))
    pred_day = pred_day[:, 1]
    preds[:, day] = pred_day
    #preds.append(preds)

0 0.7837751383827988
1 0.6885343940363272
2 0.6145447970849212
3 0.5542961252816333
4 0.515458540791633
5 0.4975313621317905
6 0.47240688715195683
7 0.45634335623487526
8 0.44684431587438456
9 0.4542432755695252
10 0.4464618508525493
11 0.4447233734805708
12 0.450856721648911
13 0.4436107479625045
14 0.43165697755278015
15 0.4160106812049734
16 0.42239437011487857
17 0.4305860754916414
18 0.46823454145920834
19 0.522906177853189
20 0.512704792634419
21 0.5167728296848488
22 0.5412714528107703
23 0.5671330421963228
24 0.5931615254095852
25 0.6196141971016105
26 0.6302189090706796
27 0.6378403938694334
28 0.6478053461656144
29 0.6695502211343217


In [225]:
preds_norm = (preds/preds.sum(axis=1)[:,None]).round(4)
pred_rps = rps(preds_norm, ground_truth)
pred_rps

3.997514963750313

In [226]:
preds_norm

array([[0.0005, 0.002 , 0.0059, ..., 0.0445, 0.0492, 0.0497],
       [0.0155, 0.0291, 0.0389, ..., 0.0346, 0.0212, 0.0164],
       [0.0012, 0.0023, 0.006 , ..., 0.0439, 0.0359, 0.0305],
       ...,
       [0.0435, 0.0477, 0.0525, ..., 0.0095, 0.008 , 0.0115],
       [0.011 , 0.0226, 0.0234, ..., 0.0327, 0.0277, 0.0278],
       [0.063 , 0.0554, 0.0407, ..., 0.042 , 0.0347, 0.019 ]])